In [2]:
import pandas as pd
import os

In [3]:
# change directory to where dataset is located
os.chdir('../datasets')
ROOT_DIR = os.getcwd()

In [4]:
# Modifying Wiki Data: Filter to only include WIKI one month
start_date = '2017-01-01'
end_date = '2021-12-31'

def _mask(dataset, start_date, end_date):
    mask = (dataset.index >= start_date) & (dataset.index <= end_date)
    return dataset.loc[mask]  

In [5]:
# BITCOIN PRICE INDEX
os.chdir(os.path.join(ROOT_DIR, 'BPI\\'))

BPI = pd.read_csv("./bpi.csv",  index_col="Date", parse_dates=True)
BPI.rename(columns={'Closing Price (USD)': 'close',
           '24h Open (USD)': 'open', '24h High (USD)': 'high', '24h Low (USD)': 'low'}, inplace=True)
BPI.drop(['Currency'], axis=1, inplace=True)

HASH = pd.read_csv("./hash-rate.csv",  index_col="Timestamp", parse_dates=True)
MINE_DIFF = pd.read_csv("./difficulty.csv",  index_col="Timestamp", parse_dates=True)
COST_PER_TRANS = pd.read_csv("./cost-per-transaction.csv", index_col="Timestamp", parse_dates=True)
OUPUT = pd.read_csv("./output-volume.csv", index_col="Timestamp", parse_dates=True)
EXTV = pd.read_csv("./estimated-transaction-volume-usd.csv", index_col="Timestamp", parse_dates=True)
TRADE_VOL = pd.read_csv("./trade-volume.csv", index_col="Timestamp", parse_dates=True)
NTRANS = pd.read_csv("./n-transactions.csv", index_col="Timestamp", parse_dates=True)

BPI = _mask(BPI, start_date, end_date)
HASH = _mask(HASH, start_date, end_date)
EXTV = _mask(EXTV, start_date, end_date)
NTRANS = _mask(NTRANS, start_date, end_date)
MINE_DIFF = _mask(MINE_DIFF, start_date, end_date)
COST_PER_TRANS = _mask(COST_PER_TRANS, start_date, end_date)
OUPUT = _mask(OUPUT, start_date, end_date)
TRADE_VOL = _mask(TRADE_VOL, start_date, end_date)


In [6]:
# MACRO
os.chdir(ROOT_DIR)

GP = pd.read_csv("./gold-price.csv",  index_col="Date", parse_dates=True)
GP.dropna(axis=1, how="all", inplace=True)
GP.drop(['Open', 'High', 'Low', 'Change %', 'Vol.'], axis=1, inplace=True)
GP.rename(columns={'Price': 'Gold price'}, inplace=True)
# conver string to int
for i in range(len(GP["Gold price"])):
    GP["Gold price"][i] = "".join(GP["Gold price"][i].split(","))
GP["Gold price"] = pd.to_numeric(GP["Gold price"])


USD_CNY = pd.read_csv("./usd-cny.csv",  index_col="Date", parse_dates=True)
USD_CNY.drop(['Open', 'High', 'Low', 'Change %'], axis=1, inplace=True)
USD_CNY.rename(columns={'Price': 'USD-CNY Price'}, inplace=True)

GP = _mask(GP, start_date, end_date)
USD_CNY = _mask(USD_CNY, start_date, end_date)

In [7]:
# MEDIA AND INTERESTS
SVI = pd.read_csv("./svi.csv", index_col="Week", parse_dates=True)
# SVI.rename(columns={'Category: All categories':'SVI'}, inplace=True)

WIKI = pd.read_csv("./wikishark.csv", index_col="DateTime", parse_dates=True)
WIKI.rename(columns={'Bitcoin[en]':'Wikiviews'}, inplace=True)

SVI = _mask(SVI, start_date, end_date)
WIKI = _mask(WIKI, start_date, end_date)

In [10]:
df = pd.concat([BPI,EXTV, NTRANS, HASH, MINE_DIFF, COST_PER_TRANS,
     GP, OUPUT, TRADE_VOL, USD_CNY, SVI, WIKI], axis=1)

# 7th differencing, (daily to weekly freq)
# df_resampled = df.resample('7D').mean().interpolate()
df_resampled = df.copy()
df_resampled.fillna(df_resampled.mean(), inplace=True)
df_resampled.head()

,close,open,high,low,estimated-transaction-volume-usd,n-transactions,hash-rate,difficulty,cost-per-transaction,Gold price,output-volume,trade-volume,USD-CNY Price,SVI,Wikiviews
2017-01-01,998.079443,966.567909,1005.074602,961.719392,1.667890e+08,180502.0,2.463611e+06,3.176884e+11,10.914479,1377.316553,1.084341e+06,3.989748e+06,6.72991,7.000000,3139
2017-01-02,1018.369458,998.079638,1033.051602,994.997792,2.799811e+08,290951.0,2.526780e+06,3.176884e+11,7.350606,1294.600000,1.573982e+06,6.822471e+06,6.94500,18.574713,9165
2017-01-03,1036.750461,1018.367417,1037.211425,1014.576142,3.417543e+08,301664.0,2.589950e+06,3.176884e+11,7.415466,1295.600000,1.950525e+06,8.324526e+06,6.96100,18.574713,12354
2017-01-04,1137.239897,1036.750186,1146.956329,1036.327096,4.336781e+08,328642.0,2.432026e+06,3.176884e+11,7.000531,1287.300000,2.482038e+06,9.340584e+06,6.93510,18.574713,10757
2017-01-05,1002.860000,1137.239606,1165.785631,883.480930,5.657800e+08,288501.0,2.210933e+06,3.176884e+11,6.913398,1287.400000,2.998215e+06,2.387011e+07,6.89000,18.574713,11938


In [11]:
print("Before sampling NaN values: ",len(df[df.isna().any(axis=1)]))
print("After sampling NaN values: ",len(df_resampled[df_resampled.isna().any(axis=1)]))

Before sampling NaN values:  1826
After sampling NaN values:  0


In [12]:
df_resampled[df_resampled.isna().any(axis=1)]

,close,open,high,low,estimated-transaction-volume-usd,n-transactions,hash-rate,difficulty,cost-per-transaction,Gold price,output-volume,trade-volume,USD-CNY Price,SVI,Wikiviews


In [13]:
df_resampled.info()
df_resampled.describe()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1826 entries, 2017-01-01 to 2021-12-31
Freq: D
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   close                             1826 non-null   float64
 1   open                              1826 non-null   float64
 2   high                              1826 non-null   float64
 3   low                               1826 non-null   float64
 4   estimated-transaction-volume-usd  1826 non-null   float64
 5   n-transactions                    1826 non-null   float64
 6   hash-rate                         1826 non-null   float64
 7   difficulty                        1826 non-null   float64
 8   cost-per-transaction              1826 non-null   float64
 9   Gold price                        1826 non-null   float64
 10  output-volume                     1826 non-null   float64
 11  trade-volume                      1826 non-

,close,open,high,low,estimated-transaction-volume-usd,n-transactions,hash-rate,difficulty,cost-per-transaction,Gold price,output-volume,trade-volume,USD-CNY Price,SVI,Wikiviews
count,1826.000000,1826.000000,1826.000000,1826.000000,1.826000e+03,1826.000000,1.826000e+03,1.826000e+03,1826.000000,1826.000000,1.826000e+03,1.826000e+03,1826.000000,1826.000000,1826.000000
mean,15470.656119,15446.307986,15900.046760,14932.582596,2.004009e+09,282361.699343,7.478909e+07,1.034516e+13,72.066568,1377.316553,1.858920e+06,3.828892e+08,6.729910,18.574713,8577.153888
std,16954.387251,16943.040894,17426.130778,16377.809151,2.112128e+09,55605.460107,5.422990e+07,7.524769e+12,58.827950,217.855991,1.334892e+06,4.398635e+08,0.203799,5.753845,9960.120160
min,778.839780,778.839783,822.860239,748.698368,1.121429e+08,124640.000000,2.147763e+06,3.176884e+11,5.612592,867.400000,4.219402e+05,0.000000e+00,6.269000,4.000000,1538.000000
25%,5515.977113,5506.636185,5643.917919,5281.151540,6.858568e+08,241893.500000,2.620610e+07,3.511061e+12,30.678315,1262.775000,1.019258e+06,1.043191e+08,6.614950,18.574713,3402.500000
50%,8660.351137,8646.125856,8844.803315,8348.784186,1.163623e+09,287523.000000,6.206519e+07,7.934713e+12,51.440029,1377.316553,1.553934e+06,2.406228e+08,6.729910,18.574713,5924.000000
75%,14159.005555,14132.352920,14863.570297,13345.497015,2.403180e+09,322992.250000,1.216233e+08,1.686567e+13,97.636480,1536.275000,2.310978e+06,5.018060e+08,6.878675,18.574713,9782.500000
max,67553.948930,67554.840000,68990.900000,66316.000000,1.464262e+10,490644.000000,1.985140e+08,2.504649e+13,300.310549,1888.700000,2.452867e+07,4.956850e+09,7.178600,100.000000,131165.000000


In [14]:
df_resampled.columns

Index(['close', 'open', 'high', 'low', 'estimated-transaction-volume-usd',
       'n-transactions', 'hash-rate', 'difficulty', 'cost-per-transaction',
       'Gold price', 'output-volume', 'trade-volume', 'USD-CNY Price', 'SVI',
       'Wikiviews'],
      dtype='object')

In [10]:
# SAVE DATASET
df_resampled.to_csv("../complete-merged-df.csv")